In [23]:
import pandas as pd
import numpy as np
import json
import os
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

# Load json files for train, test and gold scores
# reference: https://stackoverflow.com/questions/50475635/loading-jsonl-file-as-json-objects
def load_jsonl_file(file_path):
    with open(file_path, 'r') as f:
        return pd.DataFrame([json.loads(line) for line in f])

train_df = load_jsonl_file('/train/subtaskA_dev_monolingual.jsonl')
test_df = load_jsonl_file('/test/subtaskA_monolingual.jsonl')
scores_df = load_jsonl_file('/gold_scores/subtaskA_monolingual.jsonl')

# sample 5% of the train dataset
train_df = train_df.sample(frac=0.05, replace=True, random_state=1)

# Universal Sentence Encoder
usi = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Define batch size
batch_size = 32
# Function to generate embeddings
def generate_embeddings(text_list):
    embeddings_list = []
    for i in range(0, len(text_list), batch_size):
        batch = text_list[i:i+batch_size]
        embeddings = usi(batch).numpy()
        embeddings_list.extend(embeddings)
    return np.array(embeddings_list)

# Apply USE embeddings to train and test sets
X_train = generate_embeddings(train_df['text'].tolist())
y_train = train_df['label']
X_test = generate_embeddings(test_df['text'].tolist())

# SVM training
svm_classifier = SVC(kernel='sigmoid', C=100)
svm_classifier.fit(X_train, y_train)
svm_y_pred = svm_classifier.predict(X_test)

# Random Forest training
forest_classifier = RandomForestClassifier(n_estimators=5000,
    max_features="log2",
    max_depth=500,
    max_leaf_nodes=500)
forest_classifier.fit(X_train, y_train)
forest_y_pred = forest_classifier.predict(X_test)

# Save results to JSONL
predictions_svm = pd.DataFrame({
    'id': test_df['id'].astype(int),
    'label': svm_y_pred.astype(int)
})

predictions_svm.to_json('/svm/subtaskA_monolingual.jsonl', orient='records', lines=True, force_ascii=False)

predictions_rf = pd.DataFrame({
    'id': test_df['id'].astype(int),
    'label': forest_y_pred.astype(int)
})

predictions_rf.to_json('/forest/subtaskA_monolingual.jsonl', orient='records', lines=True, force_ascii=False)

# Print accuracies
print("Accuracy with SVM:", metrics.accuracy_score(scores_df['label'].astype(int), predictions_svm['label'].astype(int)))
print("Accuracy with Random Forest:", metrics.accuracy_score(scores_df['label'].astype(int), predictions_rf['label'].astype(int)))

Accuracy with SVM: 0.6355333800186741
Accuracy with Random Forest: 0.36528361344537813
